In [10]:
from itertools import permutations
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM

In [11]:
def get_candidates_setwise(Ci,Ak,Sk):
    Ci_final = set()
    for cj in Ci:
        # Ci.remove(cj)
        for a in Ak:
            c0j = cj.split()
            for i, word in zip(Sk, a):
                if type(word) == int:
                    continue
                    # print(Ak,Sk)
                    # print('i',i)
                    # print('word',word)
                c0j[i] = word
                # print(c0j)
            Ci_final.add(' '.join((c0j)))
    return Ci_final

In [12]:
from itertools import permutations

def candidate_set_generation(xi, pi0, Ti, T0i, F):
    # Initialize candidate sentence with masked tokens
    c0 = ' '.join(['<mask>'] * len(xi))
    # print(f'pi0 {pi0}')
    # print(f' c0 {c0}' )
    Ci = {c0}
    # Ci = [c0]
    # print(f'Ci {Ci}')
    Tshared = Ti.intersection(T0i)
    #print(f'T shared {Tshared}')

    for tk in Tshared:
        #print(tk)
        Wk = [word for word, pos in xi if pos == tk]
        # print(f'Wk {Wk}')
    
        Sk = [i for i, pos in enumerate(pi0) if pos[1] == tk]
        Ak = []

        # Sk=[0]
        # print(f'Sk {Sk}')
        #TODO: Change the logic here
        # print(f'len(Wk) {len(Wk)} len(Sk) {len(Sk)}')
        # Ex: Wk = ['a'] Sk = [0,1] or Sk=[1,2,3]
        if len(Wk)==1 and len(Sk)>1:
            for i in Sk:
                Ak.append((Wk[0],i))
        elif len(Sk)==1: 
            for word in Wk:
                Ak.append((word,Sk[0]))
        #Ex: Wk = ['a','b'] Sk = [0,1] or Sk=[1,2,3]
        elif len(Wk)<=len(Sk):
            for word in Wk:
                for i in Sk:
                    Ak.append((word,i))
        else:
            Ak = list(permutations(Wk,len(Sk)))
            # print(f'Ak {Ak}')
        # print(f'Ak {Ak}')
        Ci_tag = get_candidates_setwise(Ci,Ak,Sk)
        Ci = Ci.union(Ci_tag)
    return Ci


In [17]:

# offensive sentence 
# xi = 'i get this feeling that derailments happen a lot and we are only paying attention now because of what happened in ohio that said why the fuck do they happen so often'
xi = 'here comes the beginning of another fucking recession'
# offensive sentence pos sequence 
# tag_xi = ['PRP', 'VBP', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'DT', 'NN', 'CC', 'PRP', 'VBP', 'RB', 'VBG', 'NN', 'RB', 'IN', 'IN', 'WP', 'VBD', 'IN', 'NNP', 'WDT', 'VBD', 'WRB', 'DT', 'BW', 'VBP', 'PRP', 'VB', 'RB', 'RB']
tag_xi = ['RB', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'BW', 'NN']

# non-offensive sentence
# pi0 = 'it is not they happen all the time there are thousands of train derailments a year but when a particularly bad one happens news outlets jump on the trend and start reporting more they are not more frequent this is just the first a lot of people are hearing about them edit you all can downvote but i am right look it up if you are really concerned you should know this'
pi0 = 'this was predicted years ago on the superstonk sub once they start closing that whole recession will kick in and the market will bleed greedy money stealing criminals have bled the citizens dry and still want more'
pi0 = pi0.split(' ')
# non-offensive sentence pos tags 
# tag_pi0 = ['PRP', 'VBZ', 'RB', 'PRP', 'VBP', 'PDT', 'DT', 'NN', 'EX', 'VBP', 'NNS', 'IN', 'NN', 'NNS', 'DT', 'NN', 'CC', 'WRB', 'DT', 'RB', 'JJ', 'NN', 'VBZ', 'NN', 'NNS', 'VB', 'IN', 'DT', 'NN', 'CC', 'VBP', 'VBG', 'JJR', 'PRP', 'VBP', 'RB', 'RBR', 'JJ', 'DT', 'VBZ', 'RB', 'DT', 'JJ', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'VBG', 'IN', 'PRP', 'NN', 'PRP', 'DT', 'MD', 'VB', 'CC', 'PRP', 'VBP', 'JJ', 'VB', 'PRP', 'RP', 'IN', 'PRP', 'VBP', 'RB', 'JJ', 'PRP', 'MD', 'VB', 'DT']
tag_pi0 = ['DT', 'VBD', 'VBN', 'NNS', 'RB', 'IN', 'DT', 'NN', 'NN', 'IN', 'PRP', 'VBP', 'VBG', 'DT', 'JJ', 'NN', 'MD', 'VB', 'RP', 'CC', 'DT', 'NN', 'MD', 'VB', 'JJ', 'NN', 'VBG', 'NNS', 'VBP', 'VBN', 'DT', 'NNS', 'JJ', 'CC', 'RB', 'VBP', 'JJR']
tag_pi0 = tag_pi0

# unique pos tags/tokens of non-offensive sentence
T0i = set(tag_pi0)
# unique pos tags/tokens of offensive sentence
Ti = set(tag_xi)


# tuple set of word in offensive sentence and its corresponding pos tag
xi_mod = [(word,tag) for word,tag in zip(xi.split(),tag_xi)]
# tuple set of word in non-offensive sentence and its corresponding pos tag
pi0_mod = [(word,tag) for word,tag in zip(pi0,tag_pi0)]
candidate_sentences = candidate_set_generation(xi=xi_mod,pi0=pi0_mod,Ti=Ti,T0i=T0i,F=None)

In [18]:
model = RobertaForMaskedLM.from_pretrained('distilroberta-base')
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

# sequence = f"What the {tokenizer.mask_token} is going on?" # "The world will end in <mask>"
# sequence = f"they <mask> so i happen <mask> this attention <mask> are <mask> that feeling <mask> a"
def fill_mask_topk(sequence, k=5):
    input_seq = tokenizer.encode(sequence, return_tensors='pt') # tensor([[0, 133, 232, 40, 253, 11, 50264, 2]])
    mask_token_index = torch.where(input_seq == tokenizer.mask_token_id)[1] # (tensor([0]), tensor([6])) - we only want the the 2nd dimension
    token_logits = model(input_seq).logits
    masked_token_logits = token_logits[0, mask_token_index, :]
    mask_filled_sentences=[]
    top_k_tokens = torch.topk(masked_token_logits, k, dim=1).indices[0].tolist()
    for token in top_k_tokens:
        mask_filled_sentences.append(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))
    return mask_filled_sentences


In [20]:
mask_filled_candidate_sentences=[]
for i,sentence in enumerate(list(candidate_sentences)[-50:-1]):
    print(f'{i+1}. {sentence}')
    mask_filled_candidate_sentences.append(fill_mask_topk((sentence),k=5))
print(mask_filled_candidate_sentences)

1. another <mask> <mask> <mask> <mask> of <mask> beginning
2. <mask> <mask> <mask> <mask> here of <mask> recession
3. the <mask> <mask> <mask> <mask> of <mask> <mask>
4. the <mask> <mask> <mask> <mask> <mask> <mask> beginning
5. <mask> <mask> <mask> <mask> here of <mask> <mask>
6. <mask> <mask> <mask> <mask> <mask> of <mask> recession
7. the <mask> <mask> <mask> <mask> <mask> <mask> recession
8. <mask> <mask> <mask> <mask> here <mask> <mask> <mask>
9. another <mask> <mask> <mask> here of <mask> beginning
10. another <mask> <mask> <mask> here of <mask> recession
11. another <mask> <mask> <mask> here <mask> <mask> recession
12. the <mask> <mask> <mask> here of <mask> <mask>
13. <mask> <mask> <mask> <mask> <mask> of <mask> <mask>
14. the <mask> <mask> <mask> here of <mask> recession
15. another <mask> <mask> <mask> here <mask> <mask> <mask>
16. another <mask> <mask> <mask> <mask> <mask> <mask> recession
17. <mask> <mask> <mask> <mask> <mask> of <mask> beginning
18. <mask> <mask> <mask> <m

In [89]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer

# Load the pre-trained DistilRoBERTa model and tokenizer
model_name = "distilroberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the masked sentence
sentence = "The quick brown <mask> jumps over the lazy dog."

# Initialize the pipeline for masked language modeling
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

# Use the pipeline to fill in the masked token
results = fill_mask(sentence)
print(results)
# Print the top 5 predictions
for result in results[:5]:
    print(result['sequence'])


[{'score': 0.06604349613189697, 'token': 23602, 'token_str': ' fox', 'sequence': 'The quick brown fox jumps over the lazy dog.'}, {'score': 0.06038545444607735, 'token': 324, 'token_str': 'ie', 'sequence': 'The quick brownie jumps over the lazy dog.'}, {'score': 0.04486118257045746, 'token': 4758, 'token_str': ' cat', 'sequence': 'The quick brown cat jumps over the lazy dog.'}, {'score': 0.036140426993370056, 'token': 26882, 'token_str': ' Labrador', 'sequence': 'The quick brown Labrador jumps over the lazy dog.'}, {'score': 0.034533023834228516, 'token': 13182, 'token_str': ' buck', 'sequence': 'The quick brown buck jumps over the lazy dog.'}]
The quick brown fox jumps over the lazy dog.
The quick brownie jumps over the lazy dog.
The quick brown cat jumps over the lazy dog.
The quick brown Labrador jumps over the lazy dog.
The quick brown buck jumps over the lazy dog.


In [21]:
mask_filled_candidate_sentences=[]
for i,sentence in enumerate(list(candidate_sentences)[-50:-1]):
    print(f'{i+1}. {sentence}')
    number_of_masked_tokens = sentence.count('<mask>')
    for i in range(number_of_masked_tokens):
        sentence=fill_mask(sentence.replace("<mask>", tokenizer.mask_token), top_k=1)[0][0]['sequence']
        print(sentence)
    # mask_filled_candidate_sentences.append([result['sequence'] for result in results])
    break
# print(mask_filled_candidate_sentences)

1. another <mask> <mask> <mask> <mask> of <mask> beginning
<s>another ep<mask><mask><mask> of<mask> beginning</s>
<s><s>another ephemer<mask><mask> of<mask> beginning</s></s>
<s><s><s>another ephemeral<mask> of<mask> beginning</s></s></s>
<s><s><s><s>another ephemeral tale of<mask> beginning</s></s></s></s>


KeyError: 0

In [6]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer

# Load the pre-trained DistilRoBERTa model and tokenizer
model_name = "distilroberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the sentence with multiple masks
sentence = "The <mask> brown <mask> jumps over the lazy <mask>."

# Initialize the pipeline for masked language modeling
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

# Use the pipeline to fill in the first masked token
results_1 = fill_mask(sentence.replace("[MASK]", tokenizer.mask_token), top_k=5)
sentence = results_1[0][0]['sequence']
print(sentence)

# Use the pipeline to fill in the second masked token
results_2 = fill_mask(sentence.replace("[MASK]", tokenizer.mask_token), top_k=5)
sentence = results_2[0][0]['sequence']
print(sentence)
# Use the pipeline to fill in the third masked token
results_3 = fill_mask(sentence.replace("[MASK]", tokenizer.mask_token), top_k=5)
sentence = results_3[0]['sequence']
print(sentence)
# Print the top predictions for each mask
# for result in results_1:
    # print(result['sequence'].replace(tokenizer.mask_token, "quick"))

# for result in results_2:
#     print(result['sequence'].replace(tokenizer.mask_token, "fox"))


<s>The golden brown<mask> jumps over the lazy<mask>.</s>
<s><s>The golden brown cat jumps over the lazy<mask>.</s></s>
The golden brown cat jumps over the lazy couch.
